[生成器使用入门协程](https://zhuanlan.zhihu.com/p/37049161)

[深入理解yield from语法](https://zhuanlan.zhihu.com/p/37264442)


# 生成器与协程

## 生成器
https://www.liaoxuefeng.com/wiki/0014316089557264a6b348958f449949df42a6d3a2e542c000/0014317799226173f45ce40636141b6abc8424e12b5fb27000

通过列表生成式，我们可以直接创建一个列表。但是，受到内存限制，列表容量肯定是有限的。而且，创建一个包含100万个元素的列表，不仅占用很大的存储空间，如果我们仅仅需要访问前面几个元素，那后面绝大多数元素占用的空间都白白浪费了。

所以，如果列表元素可以按照某种算法推算出来，那我们是否可以在循环的过程中不断推算出后续的元素呢？这样就不必创建完整的list，从而节省大量的空间。在Python中，这种一边循环一边计算的机制，称为生成器：generator。

### 创建 generator 的两种方式
#### 第一种：把一个列表生成式的[]改成()

In [9]:
L = [x * x for x in range(5)]
g = (x * x for x in range(5))
print(L)
print(g)

[0, 1, 4, 9, 16]
<generator object <genexpr> at 0x00A09ED0>


创建L和g的区别仅在于最外层的[]和()，L是一个list，而g是一个generator。

我们可以直接打印出list的每一个元素，但我们怎么打印出generator的每一个元素呢？

如果要一个一个打印出来，可以通过next()函数获得generator的下一个返回值：

##### 通过next()函数获得generator的下一个返回值

In [10]:
print(next(g))
print(next(g))
print(next(g))
print(next(g))
print(next(g))
print(next(g))

0
1
4
9
16


StopIteration: 

**generator保存的是算法**，每次调用next(g)，就计算出g的下一个元素的值，直到计算到最后一个元素，没有更多的元素时，抛出StopIteration的错误。

当然，上面这种不断调用next(g)实在是太变态了，正确的方法是使用for循环，因为generator也是可迭代对象：

In [11]:
g = (x * x for x in range(5))
for i in g:
    print(i)

0
1
4
9
16


所以，我们创建了一个generator后，基本上永远不会调用next()，而是通过for循环来迭代它，并且不需要关心StopIteration的错误。


如果一个函数定义中包含yield关键字，那么这个函数就不再是一个普通函数，而是一个generator

以斐波拉契数为例，可以输出斐波那契数列的前N个数

In [12]:
def fib(max):
    n, a, b = 0, 0, 1
    while n < max:
        print(b)
        a, b = b, a + b
        n = n + 1
    return 'done'

fib(4)

1
1
2
3


'done'

#### 第二种：用 yield 

仔细观察，可以看出，fib函数实际上是定义了斐波拉契数列的推算规则，可以从第一个元素开始，推算出后续任意的元素，这种逻辑其实非常类似generator。

也就是说，上面的函数和generator仅一步之遥。要把fib函数变成generator，只需要把print(b)改为yield b就可以了：

In [18]:
def fib(max):
    n, a, b = 0, 0, 1
    while n < max:
        yield b
        a, b = b, a + b
        n = n + 1
    return 'done'

f = fib(4)
print(f)
for i in f:
    print(i)

<generator object fib at 0x00A049C0>
1
1
2
3


这里，最难理解的就是generator和函数的执行流程不一样。函数是顺序执行，遇到return语句或者最后一行函数语句就返回。
**而变成generator的函数，在每次调用 next()的时候执行，遇到yield语句返回，再次执行时从上次返回的yield语句处继续执行。**


### 调用生成器

调用该generator时，首先要生成一个generator对象，然后用next()函数不断获得下一个返回值：

In [19]:
def odd():
    print('step 1')
    yield 1
    print('step 2')
    yield(3)
    print('step 3')
    yield(5)
    
o = odd()
print(next(o))
print(next(o))
print(next(o))
print(next(o))

step 1
1
step 2
3
step 3
5


StopIteration: 

odd不是普通函数，而是generator，在执行过程中，遇到yield就中断，下次又继续执行。执行3次yield后，已经没有yield可以执行了，所以，第4次调用next(o)就报错。

In [22]:
def fib(max):
    n, a, b = 0, 0, 1
    while n < max:
        yield b
        a, b = b, a + b
        n = n + 1
    return 'done'

g = fib(4)
for n in fib(4):
    print(n)

1
1
2
3


**但是用for循环调用generator时，发现拿不到generator的return语句的返回值。**

如果想要拿到返回值，必须捕获StopIteration错误，返回值包含在StopIteration的value中：

In [29]:
def fib(max):
    n, a, b = 0, 0, 1
    while n < max:
        yield b
        a, b = b, a + b
        n = n + 1
    return 'done'

g = fib(4)

print(next(g))
print(next(g))
print(next(g))
print(next(g))
print(next(g))

1
1
2
3


StopIteration: done

In [31]:
def fib(max):
    n, a, b = 0, 0, 1
    while n < max:
        yield b
        a, b = b, a + b
        n = n + 1
    return 'done'

g = fib(4)
while True:
    try:
        x = next(g)
        print(x)
    except StopIteration as e:
        print('Generator return value:',e.value)
        break

1
1
2
3
Generator return value: done


### yield 与 return
http://python.jobbole.com/81911/

#### 没有 return
在一个生成器中，如果没有return，则默认执行到函数完毕时返回StopIteration；

In [2]:
def g1():
    yield 1

g = g1()
print(next(g))
print(next(g))

1


StopIteration: 

- 第一次调用next(g)时，会在执行完yield语句后挂起，所以此时程序并没有执行结束。
- 程序试图从yield语句的下一条语句开始执行，发现已经到了结尾，所以抛出StopIteration异常。

#### 有return
如果遇到return,如果在执行过程中 return，则直接抛出 StopIteration 终止迭代。

In [3]:
def g2():
    yield 'a'
    return
    yield 'b'

g = g2()
print(next(g))
print(next(g))

a


StopIteration: 

- 程序停留在执行完yield 'a'语句后的位置。
- 程序发现下一条语句是return，所以抛出StopIteration异常，这样yield 'b'语句永远也不会执行。

#### 有带返回值的 return

如果在return后返回一个值，那么这个值为StopIteration异常的说明，不是程序的返回值。

生成器没有办法使用return来返回值。

**如果想要拿到返回值，必须捕获StopIteration错误，返回值包含在StopIteration的value中**

In [28]:
def g3():
    yield 'hello'
    return 'world'

g = g3()
print(next(g))
print(next(g))

hello


StopIteration: world

In [31]:
def g3():
    yield 'hello'
    return 'world'

try:
    g = g3()
    print(next(g))
    print(next(g))
except StopIteration as e:
    print('Generator return value:',e.value)


hello
Generator return value: world


### 生成器支持的方法

- close() 手动关闭生成器函数，后面的调用会直接返回StopIteration异常。
- send() 生成器函数最大的特点是可以接受外部传入的一个变量，并根据变量内容计算结果后返回。这是生成器函数最难理解的地方，也是最重要的地方，实现后面我会讲到的协程就全靠它了。

- throw() 用来向生成器函数送入一个异常，可以结束系统定义的异常，或者自定义的异常。

In [8]:
help(g)

Help on generator object:

g3 = class generator(object)
 |  Methods defined here:
 |  
 |  __del__(...)
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __iter__(self, /)
 |      Implement iter(self).
 |  
 |  __next__(self, /)
 |      Implement next(self).
 |  
 |  __repr__(self, /)
 |      Return repr(self).
 |  
 |  close(...)
 |      close() -> raise GeneratorExit inside generator.
 |  
 |  send(...)
 |      send(arg) -> send 'arg' into generator,
 |      return next yielded value or raise StopIteration.
 |  
 |  throw(...)
 |      throw(typ[,val[,tb]]) -> raise exception in generator,
 |      return next yielded value or raise StopIteration.
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  gi_code
 |  
 |  gi_frame
 |  
 |  gi_running
 |  
 |  gi_yieldfrom
 |      object being iterated by yield from, or None



#### close() 
手动关闭生成器函数，后面的调用会直接返回StopIteration异常。

In [11]:
def g4():
    yield 1
    yield 2
    yield 3

g = g4()
print(next(g))
g.close()
print(next(g))

1


StopIteration: 

#### send()

生成器函数最大的特点是可以接受外部传入的一个变量，并根据变量内容计算结果后返回。
这是生成器函数最难理解的地方，也是最重要的地方，实现后面我会讲到的协程就全靠它了。

In [76]:
def gen():
    value = 0
    while True:
        receive = yield value
        if receive == 'e':
            break
        value = 'got:{}'.format(receive)
        
g = gen()  # 调用生成器时，首先要生成一个generator对象
print(g.send(None))
print(g.send('aaa'))
print(g.send(3))
print(g.send('e'))

0
got:aaa
got:3


StopIteration: 

其实receive=yield value包含了3个步骤： 
1、向函数外抛出（返回）value 
2、暂停(pause)，等待next()或send()恢复 
3、赋值receive=MockGetValue() 。 这个MockGetValue()是假想函数，用来接收send()发送进来的值


执行流程：

- **通过g.send(None)或者next(g)可以启动生成器函数**，并执行到第一个yield语句结束的位置。此时，执行完了yield语句，但是没有给receive赋值。yield value会输出初始值0
**注意：在启动生成器函数时只能send(None),如果试图输入其它的值都会得到错误提示信息。**
- 通过g.send(‘aaa’)，会传入aaa，并赋值给receive，然后计算出value的值，并回到while头部，执行yield value语句有停止。此时yield value会输出”got: aaa”，然后挂起。
- 通过g.send(3)，会重复第2步，最后输出结果为”got: 3″
当我们g.send(‘e’)时，程序会执行break然后推出循环，最后整个函数执行完毕，所以会得到StopIteration异常。

从上面可以看出， 在第一次send(None)启动生成器（执行1–>2，通常第一次返回的值没有什么用）之后，对于外部的每一次send()，生成器的实际在循环中的运行顺序是3–>1–>2，也就是先获取值，然后dosomething，然后返回一个值，再暂停等待。


#### throw()

用来向生成器函数送入一个异常，可以结束系统定义的异常，或者自定义的异常。

致使生成器在暂停的 yield 表达式处抛出指定的异常。如果生成器处理了抛出的异常，代码会向前执行到下一个 yield 表达式，而产出的值会成为调用 generator.throw方法得到的返回值。如果生成器没有处理抛出的异常，异常会向上冒泡，传到调用方的上下文中

In [1]:
def gen():
    while True:
        try:
            yield 'normal value'
            yield 'normal value 2'
            print('here')
        except ValueError:
            print('we got ValueError here')
        except TypeError:
            break
            
g = gen()
print(next(g))
print(g.throw(ValueError))
print(next(g))
print(g.throw(TypeError))

normal value
we got ValueError here
normal value
normal value 2


StopIteration: 

解释：

- print(next(g))：会输出normal value，并停留在yield ‘normal value 2’之前。
- 由于执行了g.throw(ValueError)，所以会跳过所有后续的try语句，也就是说yield ‘normal value 2’不会被执行，然后进入到except语句，打印出we got ValueError here。然后再次进入到while语句部分，消耗一个yield，所以会输出normal value。
- print(next(g))，会执行yield ‘normal value 2’语句，并停留在执行完该语句后的位置。
- g.throw(TypeError)：会跳出try语句，从而print(‘here’)不会被执行，然后执行break语句，跳出while循环，然后到达程序结尾，所以跑出StopIteration异常。

下面给出一个综合例子，用来把一个多维列表展开，或者说扁平化多维列表)

In [9]:
def flatten(nested):
    try:
        if isinstance(nested,str):
            raise TypeError
        for sublist in nested:
            for element in flatten(sublist):
                print('got:',element)
    except TypeError:
        yield nested
        
L=['aaadf',[1,2,3],2,4,[5,[6,[8,[9]],'ddf'],7]]

for num in flatten(L):
    print(num)

got: aaadf
got: 1
got: 2
got: 3
got: 2
got: 4
got: 5
got: 6
got: 8
got: 9
got: ddf
got: 7


### 总结

- 按照鸭子模型理论，生成器就是一种迭代器，可以使用for进行迭代。
- 第一次执行next(generator)时，会执行完yield语句后程序进行挂起，所有的参数和状态会进行保存。再一次执行next(generator)时，会从挂起的状态开始往后执行。在遇到程序的结尾或者遇到StopIteration时，循环结束。
- 可以通过generator.send(arg)来传入参数，这是协程模型。
- 可以通过generator.throw(exception)来传入一个异常。throw语句会消耗掉一个yield。可以通过generator.close()来手动关闭生成器。
- next()等价于send(None)

## 流畅的python - 协程
https://segmentfault.com/a/1190000009769387

### 用作协程的生成器的基本行为

#### 最简单的协程

In [40]:
import inspect

def simple_coroutine():
    print('-> coroutine started')
    x = yield
    print('-> coroutine received:',x)
    
my_coro = simple_coroutine() # 调用函数得到生成器对象
print(inspect.getgeneratorstate(my_coro))

my_coro.send(None)
print(inspect.getgeneratorstate(my_coro))
my_coro.send(42) # 这里，控制权流动到协程定义体的末尾，导致生成器像往常一样抛出 StopIteration 异常

GEN_CREATED
-> coroutine started
GEN_SUSPENDED
-> coroutine received: 42


StopIteration: 

- 协程使用生成器函数定义：定义体中有 yield 关键字。
- yield 在表达式中使用；如果协程只需从客户那里接收数据，那么产出的值是 None ——这个值是隐式指定的，因为 yield 关键字右边没有表达式
- **与创建生成器的方式一样，调用函数得到生成器对象。**
- **send方法的参数会成为暂停yield表达式的值**，所以，仅当协程处于暂停状态是才能调用send方法。如果协程还未激活（GEN_CREATED 状态）要调用next(my_coro) 激活协程，也可以调用my_coro.send(None)
- 最先调用my_coro.send(None) / next(my_coro) 这一步通常称为”预激“（prime）协程---即，让协程向前执行到第一个yield表达式，准备好作为活跃的协程使用。
- 控制权流动到协程定义体的末尾，导致生成器像往常一样抛出 StopIteration 异常
- 注意：在启动生成器函数时只能 my_coro.send(None)/ next(my_coro) ,如果试图输入其它的值都会得到错误提示信息。

**协程四个状态**
协程可以身处四个状态中的一个。当前状态可以使用 inspect.getgeneratorstate(...) 函数确定，该函数会返回下述字符串中的一个

- GEN_CREATED # 等待开始执行
- GEN_RUNNING # 解释器正在执行（只有在多线程应用中才能看到这个状态）
- GEN_SUSPENDED # 在yield表达式处暂停
- GEN_CLOSED # 执行结束

#### 产出两个值的协程

In [83]:
def  simple_coro2(a):
    print('-> Started: a =', a)
    b = yield a
    print('-> Received: b =', b)
    c = yield a + b
    print('-> Received: c =', c)

mycoro =  simple_coro2(14) # 1
print(inspect.getgeneratorstate(mycoro))  # 2
mycoro.send(None) # 3
print(inspect.getgeneratorstate(mycoro))  # 4
mycoro.send(28) # 5
print(inspect.getgeneratorstate(mycoro)) # 6
mycoro.send(99) # 7

GEN_CREATED
-> Started: a = 14
GEN_SUSPENDED
-> Received: b = 28
GEN_SUSPENDED
-> Received: c = 99


StopIteration: 

1. 调用函数得到生成器对象
2. 协程未启动
3. 预激协程 -- 向前执行协程到第一个 yield 表达式，打印 -> Started: a = 14 消息，然后产出 a 的值，并且暂停，等待为 b 赋值
4. 协程在第一个 yield 表达式处暂停
5. 把数字28 发给暂停的协程，赋值给b。 b= MockGetValue(),这个MockGetValue()是假想函数，用来接收send()发送进来的值。打印 -> Received: b = 28 消息，产出 a + b 的值（42），然后协程暂停，等待为 c 赋值。
6. 协程在第二个 yield 表达式处暂停
7. 把数字 99 发给暂停的协程，赋值给 c。打印 -> Received: c = 99 消息，产出 a + b 的值（42）。协程到尾部，导致生成器对象抛出 StopIteration 异常

关键的一点是，协程在 yield 关键字所在的位置暂停执行。前面说过，在赋值语句中，= 右边的代码在赋值之前执行。因此，对于 b = yield a 这行代码来说，等到客户端代码再激活协程时才会设定 b 的值。这种行为要花点时间才能习惯，不过一定要理解，这样才能弄懂异步编程中 yield 的作用。


### 使用协程计算移动平均值

定义一个计算移动平均值的协程

In [89]:
def averager():
    count = 0
    total = 0.0
    average = None
    while True:
        term = yield average
        count += 1
        total += term
        average = total/count

avg = averager() # 创建协程对象
print(next(avg)) # 调用 next 函数，预激协程, 协程会向前执行到 yield 表达式，产出 average 变量的初始值——None
print(avg.send(10)) # 计算移动平均值：多次调用 .send(...) 方法，产出当前的平均值
print(avg.send(11))
print(avg.send(12))

None
10.0
10.5
11.0


- 这个无限循环表明，只要调用方不断把值发给这个协程，它就会一直接收值，然后生成结果。仅当调用方在协程上调用 .close() 方法，或者没有对协程的引用而被垃圾回收程序回收时，这个协程才会终止。
- 这里的 yield 表达式用于暂停执行协程，把结果发给调用方；还用于接收调用方后面发给协程的值，恢复无限循环

使用协程的好处是，total 和 count 声明为局部变量即可，无需使用实例属性或闭包在多次调用之间保持上下文。

如何终止执行 averager 实例（如 avg），因为定义体中有个无限循环。讨论如何终止协程之前，我们要先谈谈如何启动协程。使用协程之前必须预激，可是这一
步容易忘记。为了避免忘记，可以在协程上使用一个特殊的装饰器。接下来介绍这样一个装饰器。

### 使用装饰器预激协程

如果不预激，那么协程没什么用。调用 my_coro.send(x) 之前，记住一定要调用 next(my_coro)。为了简化协程的用法，有时会使用一个预激装饰器。示

In [96]:
from functools import wraps
from inspect import getgeneratorstate


def coroutine(func):
    '''
    装饰器： 向前执行到第一个`yield`表达式，预激`func`
    :param func: func name
    :return: primer
    '''    
    @wraps(func)
    def primer(*args,**kwargs):
        # 把装饰器生成器函数替换成这里的primer函数；调用primer函数时，返回预激后的生成器。
        gen = func(*args,**kwargs)
        # 调用被被装饰函数，获取生成器对象
        gen.send(None) # 预激生成器
        return gen # 返回生成器
    return primer 

@coroutine
def averager():
    count = 0
    total = 0.0
    average = None
    while True:
        term = yield average
        count += 1
        total += term
        average = total/count


gen = averager()  # 已经预激
print(getgeneratorstate(gen))
gen.send(10)

GEN_SUSPENDED


10.0

- 装饰器 `averager = coroutine(averager)`返回函数 primer
- `gen = averager() ` -> `gen = primer()`  
- 函数 primer 中,创建生成器对象,预激生成器,并返回生成器 gen
-  getgeneratorstate 函数指明，处于 GEN_SUSPENDED 状态，因此这个协程已经准备好，可以接收值了。

**使用 yield from 句法调用协程时，会自动预激**，因此上述示例中的@coroutine 等装饰器不兼容。Python 3.4 标准库里的 asyncio.coroutine 装饰器不会预激协程，因此能兼容 yield from 句法。

接下来探讨协程的重要特性——用于终止协程，以及在协程中抛出异常的方法。


### 终止协程和异常处理

协程中未处理的异常会向上冒泡，传给 next 函数或 send 方法的调用方（即触发协程的对象）

In [98]:
gen = averager()
print(gen.send(10))
gen.send('spam')

10.0


TypeError: unsupported operand type(s) for +=: 'float' and 'str'

使用 @coroutine 装饰器装饰的 averager 协程，可以立即开始发送值,发送的值不是数字，导致协程内部有异常抛出。由于在协程内没有处理异常，协程会终止。如果试图重新激活协程，会抛出StopIteration 异常。

从 Python 2.5 开始，客户代码可以在生成器对象上调用两个方法，显式地把异常发给协程。throw 和 close

`generator.throw(exc_type[, exc_value[, traceback]])`

致使生成器在暂停的 yield 表达式处抛出指定的异常。如果生成器处理了抛出的异常，代码会向前执行到下一个 yield 表达式，而产出的值会成为调用 generator.throw 方法得到的返回值。如果生成器没有处理抛出的异常，异常会向上冒泡，传到调用方的上下文中。

`generator.close()`

致使生成器在暂停的 yield 表达式处抛出 GeneratorExit 异常。如果生成器没有处理这个异常，或者抛出了 StopIteration 异常（通常是指运行到结尾），调用方不会报错。如果收到 GeneratorExit 异常，生成器一定不能产出值，否则解释器会抛出 RuntimeError 异常。生成器抛出的其他异常会向上冒泡，传给调用方。


In [107]:
from inspect import getgeneratorstate

class DemoException(Exception):
    """为这次演示定义的异常类型。"""
    
def demo_exc_handling():
    print('-> coroutine started')
    while True:
        try:
            x = yield
        except DemoException: # 特别处理 DemoException 异常
            print('*** DemoException handled. Continuing...')
        else:
            print('-> coroutine received: {}'.format(x)) # 如果没有异常，那么显示接收到的值
    raise RuntimeError('This line should never run.') 
    # 这一行永远不会执行 因为只有未处理的异常才会中止那个无限循环， 而一旦出现未处理的异常，协程会立即终止

    
exc_coro = demo_exc_handling()
next(exc_coro)
exc_coro.send(11)
print('--------异常传入 demo_exc_handling 不会导致协程中止------------') 
exc_coro.throw(DemoException)
print(getgeneratorstate(exc_coro))
print('--------关闭 demo_exc_handling，没有异常------------')  
exc_coro.send(11)
exc_coro.close()
print(getgeneratorstate(exc_coro))
print('--------如果无法处理传入的异常，协程会终止------------') 
exc_coro = demo_exc_handling()
next(exc_coro)
exc_coro.send(11)
exc_coro.throw(ZeroDivisionError)

-> coroutine started
-> coroutine received: 11
--------异常传入 demo_exc_handling 不会导致协程中止------------
*** DemoException handled. Continuing...
GEN_SUSPENDED
--------关闭 demo_exc_handling，没有异常------------
-> coroutine received: 11
GEN_CLOSED
--------如果无法处理传入的异常，协程会终止------------
-> coroutine started
-> coroutine received: 11


ZeroDivisionError: 

- 关闭 demo_exc_handling，没有异常
- 把 DemoException 异常传入 demo_exc_handling 不会导致协程中止
- 如果无法处理传入的异常，协程会终止

Python 3.3 引入 yield from 结构的主要原因之一与把异常传入嵌套的协程有关。另一个
原因是让协程更方便地返回值。

### 让协程返回值

这段代码会返回最终均值的结果，每次激活协程时不会产出移动平均值，而是最后一次返回。

averager 协程返回的结果是一个 namedtuple，两个字段分别是项数（count）和平均值（average）。我本可以只返回平均值，但是返回一个元组可以获得累积数据的另一个重要信息——项数

In [110]:
from collections import namedtuple

Result = namedtuple('result','count average')

def averager():
    count = 0
    total = 0.0
    average = None
    while True:
        term = yield average
        if term == None:
            break
        count += 1
        total += term
        average = total/count
    return Result(count,average)

ave = averager()
ave.send(None)
ave.send(10)
ave.send(30)
ave.send(6.5)
ave.send(None)

StopIteration: result(count=3, average=15.5)

为了返回值，协程必须正常终止；因此，这一版 averager 中有个条件判断，以便退出累计循环。

发送 None 会终止循环，导致协程结束，返回结果。一如既往，生成器对象会抛出StopIteration 异常。异常对象的 value 属性保存着返回的值。

**注意，return 表达式的值会偷偷传给调用方，赋值给 StopIteration 异常的一个属性。这样做有点不合常理，但是能保留生成器对象的常规行为——耗尽时抛出StopIteration 异常**

### 如何获取协程返回的值

#### 捕获 StopIteration 异常，获取 averager 返回的值

In [111]:
ave = averager()
ave.send(None)
ave.send(10)
ave.send(30)
ave.send(6.5)
try:
    ave.send(None)
except StopIteration as exc:
    result = exc.value
print(result)

result(count=3, average=15.5)


获取协程的返回值虽然要绕个圈子，但这是 PEP 380 定义的方式，当我们意识到这一点之后就说得通了：yield from 结构会在内部自动捕获 StopIteration 异常。这种处理方式与 for 循环处理 StopIteration 异常的方式一样：循环机制使用用户易于理解的方式处理异常。对 yield from 结构来说，解释器不仅会捕获 StopIteration 异常，还会把value 属性的值变成 yield from 表达式的值。可惜，我们无法在控制台中使用交互的方式测试这种行为，因为在函数外部使用 yield from（以及 yield）会导致句法出错。

### yield from

首先要知道，yield from 是全新的语言结构。它的作用比 yield 多很多，因此人们认为继续使用那个关键字多少会引起误解。在其他语言中，类似的结构使用 await 关键字，这个名称好多了，因为它传达了至关重要的一点：在生成器 gen 中使用 yield from subgen() 时，subgen 会获得控制权，把产出的值传给 gen 的调用方，即调用方可以直接控制 subgen。与此同时，gen 会阻塞，等待 subgen 终止。

**yield from 可用于简化 for 循环中的 yield 表达式**

In [112]:
def gen():
    for c in 'AB':
        yield c
    for i in range(1,3):
        yield i
        
list(gen())

['A', 'B', 1, 2]

可以改写为

In [113]:
def gen():
    yield from 'AB'
    yield from range(1,3)
        
list(gen())

['A', 'B', 1, 2]

**使用 yield from 链接可迭代的对象**

In [119]:
def chain(*args):
    for i in args:
        yield from i
    
s = 'AB'
t = range(3)

list(chain(s,t))

['A', 'B', 0, 1, 2]

等同于

In [118]:
def chain(*args):
    for i in args:
        for a in i:
            yield a
    
s = 'AB'
t = range(3)

list(chain(s,t))

('AB', range(0, 3))


['A', 'B', 0, 1, 2]

yield from x 表达式对x对象做的第一件事是，调用 iter(x)，获取迭代器。所以要求x是可迭代对象

可是，如果 yield from 结构唯一的作用是替代产出值的嵌套 for 循环，这个结构很有可能不会添加到 Python 语言中。yield from 结构的本质作用无法通过简单的可迭代对象说明，而要发散思维，使用嵌套的生成器。因此，引入 yield from 结构的 PEP 380 才起了“Syntax for Delegating to a Subgenerator”（“把职责委托给子生成器的句法”）这个标题。

**yield from 的主要功能是打开双向通道，把最外层的调用方与最内层的子生成器连接起来，这样二者可以直接发送和产出值，还可以直接传入异常，而不用在位于中间的协程中添加大量处理异常的样板代码。**有了这个结构，协程可以通过以前不可能的方式委托职责。

yield from 包含几个概念：

- 委派生成器

包含yield from <iterable> 表达式的生成器函数

- 子生成器

从yield from <iterable> 部分获取的生成器。

- 调用方

调用委派生成器的客户端（调用方）代码

委派生成器在 yield from 表达式处暂停时，调用方可以直接把数据发给字生成器，子生成器再把产出的值发送给调用方。子生成器返回之后，解释器会抛出StopIteration异常，并把返回值附加到异常对象上，只是委派生成器恢复。

![yield_from](yield_from.png)

这段代码从一个字典中读取男生和女生的身高和体重。然后把数据传给之前定义的 averager 协程，最后生成一个报告。

In [120]:
from collections import namedtuple


Result = namedtuple('Result', 'count average')


# 子生成器
# 这个例子和上边示例中的 averager 协程一样，只不过这里是作为字生成器使用
def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield # main 函数发送数据到这里,赋值 term
        if term is None: # 终止条件
            break
        total += term
        count += 1
        average = total/count
    return Result(count, average) # 返回的Result 会成为grouper函数中yield from表达式的值


# 委派生成器
def grouper(results, key):
     # 这个循环每次都会新建一个averager 实例，每个实例都是作为协程使用的生成器对象
    while True:
        '''grouper 发送的每个值都会经由yield from 处理，通过管道传给averager 实例。grouper会在yield from表达式处暂停，等待averager实例
        处理客户端发来的值。averager实例运行完毕后，返回的值绑定到results[key] 上。while 循环会不断创建averager实例，处理更多的值。
        '''
        results[key] = yield from averager()


# 调用方
def main(data):
    results = {}
    for key, values in data.items():
        # group 是调用grouper函数得到的生成器对象，传给grouper 函数的第一个参数是results，用于收集结果；第二个是某个键
        group = grouper(results, key) 
        next(group) # 预激 grouper 委派生成器
        for value in values:
            # 把各个value传给grouper 传入的值最终到达averager函数中 term = yield 那一行
            # grouper并不知道传入的是什么，同时grouper实例在yield from处暂停
            group.send(value) # 把None传入groupper，传入的值最终到达averager函数中，导致当前实例终止。然后继续创建下一个实例。
        group.send(None) # 如果没有group.send(None)，那么averager子生成器永远不会终止，委派生成器也永远不会在此激活，也就不会为result[key]赋值
    report(results)


# 输出报告
def report(results):
    for key, result in sorted(results.items()):
        group, unit = key.split(';')
        print('{:2} {:5} averaging {:.2f}{}'.format(result.count, group, result.average, unit))


data = {
    'girls;kg':[40, 41, 42, 43, 44, 54],
    'girls;m': [1.5, 1.6, 1.8, 1.5, 1.45, 1.6],
    'boys;kg':[50, 51, 62, 53, 54, 54],
    'boys;m': [1.6, 1.8, 1.8, 1.7, 1.55, 1.6],
}

if __name__ == '__main__':
    main(data)

 6 boys  averaging 54.00kg
 6 boys  averaging 1.68m
 6 girls averaging 44.00kg
 6 girls averaging 1.58m


## [How the heck does async/await work in Python 3.5?](https://snarky.ca/how-the-heck-does-async-await-work-in-python-3-5/)



In [1]:
def jumping_range(up_to):
    """Generator for the sequence of integers from 0 to up_to, exclusive.

    Sending a value into the generator will shift the sequence by that amount.
    """
    index = 0
    while index < up_to:
        jump = yield index
        if jump is None:
            jump = 1
        index += jump


if __name__ == '__main__':
    iterator = jumping_range(5)
    print(next(iterator))  # 0
    print(iterator.send(2))  # 2
    print(next(iterator))  # 3
    print(iterator.send(-1))  # 2
    for x in iterator:
        print(x)  # 3, 4

0
2
3
2
3
4


`yield from iterator`

等价

```
for x in iterator:
    yield x
```

In [4]:
def lazy_range(up_to):
    """Generator to return the sequence of integers from 0 to up_to, exclusive."""
    index = 0
    while index < up_to:
        yield index
        index += 1
        
if __name__ == '__main__':
    g = lazy_range(5)
    print(next(g))  # 0
    print(g.send(2))  # 2
    print(next(g))  # 3
    print(g.send(-1))  # 2

0
1
2
3


In [6]:
def lazy_range(up_to):
    """Generator to return the sequence of integers from 0 to up_to, exclusive."""
    index = 0
    def gratuitous_refactor():
        nonlocal index
        while index < up_to:
            yield index
            index += 1
    yield from gratuitous_refactor()
    
if __name__ == '__main__':
    g = lazy_range(5)
    print(next(g))  # 0

0


In [7]:
import asyncio

# Borrowed from http://curio.readthedocs.org/en/latest/tutorial.html.
@asyncio.coroutine
def countdown(number, n):
    while n > 0:
        print('T-minus', n, '({})'.format(number))
        yield from asyncio.sleep(1)
        n -= 1

loop = asyncio.get_event_loop()
tasks = [
    asyncio.ensure_future(countdown("A", 2)),
    asyncio.ensure_future(countdown("B", 3))]
loop.run_until_complete(asyncio.wait(tasks))
loop.close()

T-minus 2 (A)
T-minus 3 (B)
T-minus 1 (A)
T-minus 2 (B)
T-minus 1 (B)


事件循环调用每一个 countdown() 协程，执行直到其中某个 countdown() 协程运行到 yield from 和  asyncio.sleep(1) 处，该 countdown() 协程返回一个asyncio.Future 对象，该 asyncio.Future 对象传递给事件循环并暂停 countdown() 协程。事件循环监控 asyncio.Future 对象，直到 asyncio.sleep(1)执行完成（同时检查受到监控的其他协程）。一旦 syncio.sleep(1)执行完成，事件循环获取到之前返回 asyncio.Future 对象的 countdown() 协程，并把 asyncio.Future 对象的结果发回到最初返回 asyncio.Future 对象的 countdown() 协程，该 countdown() 协程开始再次运行。直到所有的countdown()协同程序完成运行，事件循环没有需要监控的，整个程序停止。